In [1]:
from officelib.xllib import *
from officelib.const import xlconst as xlc
from pywintypes import com_error
from scripts.tools.issuetracker import IssuetrackerAPI
from os.path import join as pjoin
import re

In [335]:
def frs_nums(s):
    pos = s.find("FRS")
    num = s[pos+3:]
    return num.split(".")

def num_tuple(s):
    return tuple(int(i) for i in frs_nums(s))
    
def sort_key(t):
    return num_tuple(t[0])

def sort_frs_item(s):
    return num_tuple(s[0])

In [389]:
def paste_data(ws, data):
    cells = ws.Cells
    cr = cells.Range
    
    di = data[0].index("Tested") + 1
    
    header_start = cr("A1")
    
    frs_start = cr("A2")
    frs_end = header_start.Offset(len(data), 1)

    id_start = frs_start.Offset(1, 3)
    id_end = frs_end.Offset(1, 3)

    tested_start = id_start.Offset(1,2)
    tested_end = id_end.Offset(1,2)

    paste_start = header_start
    paste_end = frs_end.Offset(1, len(data[0]))
    
    holdup_end = tested_end.Offset(1, 3)
    
    with screen_lock(ws.Application):
        paste_range = cr(paste_start, paste_end)
        print("Pasting test data")
        paste_range.Clear()
        paste_range.Value = data

        print("Applying alignment formatting")
        cr(frs_start, id_end).VerticalAlignment = xlc.xlTop
        cr(id_start, id_end).HorizontalAlignment = xlc.xlRight
        cr(tested_start, holdup_end).HorizontalAlignment = xlc.xlRight
        cr(tested_start, holdup_end).VerticalAlignment = xlc.xlTop

        print("Marking untested cells")
        col_os = paste_end.Column - paste_start.Column + 1
        data_iter = iter(data); next(data_iter)
        for offset, (item, level, t_id, tested, *__) in enumerate(data_iter, 1):
            row = cr(frs_start.Offset(offset, 1), frs_start.Offset(offset, col_os))
            if not tested:
                format_row(row)
            elif tested[0] == "=":
                if row[di].Value not in ("Y", "n/a"):
                    format_row(row)
            
            row[1].IndentLevel = level

        #print("Applying autofilter on untested items")
        #cr(paste_start.Offset(0, 1), paste_end).AutoFilter(Field=col_os, Criteria1="=")

        print("Applying column autofit")
        # fit after filter to account for width of filter icon
        for c in paste_range.Columns:
            c.EntireColumn.AutoFit()

def format_row(row):
    rint = row.Interior
    rint.Pattern = xlc.xlSolid
    rint.PatternColorIndex = xlc.xlAutomatic
    rint.ThemeColor = xlc.xlThemeColorAccent2
    rint.TintAndShade = 0.399975585192419
    rint.PatternTintAndShade = 0
    
def get_matrix_sheet(wb):
    matrix = None
    for ws in wb.Worksheets:
        if ws.Name == "Matrix":
            matrix = ws
            break
    if matrix is None:
        matrix = wb.Worksheets.Add()
        matrix.Name = "Matrix"
    # clear filters if present
    matrix.UsedRange.Clear()
    matrix.UsedRange.Clear()
    return matrix

In [390]:
_frs_match = re.compile(r"(.*FRS\d+)\.?([\d\.]*)").match
_toplevel_match = re.compile(r"^\>?[\+\*]{2}(FRS\d+)[\+\*]{2}.*$").match
_subitem_match = re.compile(r"^\>?[\+\*]+\s\*(FRS[\d\.]+)\:?\*\:?.*$").match
_canceled_match = re.compile(r"^[\+\*]+\s\-\*(FRS[\d\.]+)\:?\*\:?.*\-").match

In [391]:
# Misc flags for FRS item status
FRS_NA       = 1<<0  # 1
FRS_TOPLEVEL = 1<<1  # 2
FRS_SUBITEM  = 1<<2  # 4

In [392]:
def dump(node, level=0):
    for k, v in sorted(node.children.items()):
        print(" "*level + str(v.id)+ " "+str(v._tests))
        dump(v, level+1)

class Node():

    def __init__(self, id, flags, parent):
        self.id = id
        self.flags = flags
        self.parent = parent
        self.children = {}
        self._tests = []
        
    def is_na(self):
        return self.flags & FRS_NA
    
    def set_flags(self, flags):
        self.flags = flags
        
    def add_test(self, id_test):
        self._tests.append(id_test)
        for c in self.children.values():
            c.add_test(id_test)
        
    def get_tests(self):
        return sorted(self._tests)
        
    def get(self, id):
        return self.children.get(id, None)
        
    def add_child(self, id, flags=0):
        child = self.mk_child(id, flags)
        self.children[id] = child
        return child
    
    def mk_child(self, id, flags):
        return self.__class__("%s.%s"%(self.id, id), flags, self)
    
    def iter(self):
        # use .items() to sort by item order
        for _, v in sorted(self.children.items()):
            yield v
            yield from v.iter()
            
    def total_len(self):
        n = len(self.children)
        for v in self.children.values():
            n += v.total_len()
        return n
            
    def __repr__(self):
        return "%s(%r)" % (self.__class__.__name__, self.id)
    
    def is_tested(self):
        if self._tests:
            return True
        elif not self.children:
            return False
        else:
            for c in self.children.values():
                if not c.is_tested():
                    return False
            return True
        
    def is_leaf(self):
        return not self.children
    
    def has_children(self):
        return not not self.children
        
class Root(Node):
    def __init__(self):
        super().__init__("", 0, None)
        
    def mk_child(self, id, flags):
        return Node(id, flags, self)
        
    def add(self, key, flags):
        frs, nums = _frs_match(key).groups()
        child = self.get(frs)
        if not child:
            if nums:
                # Flags correspond to child in path.
                child = self.add_child(frs, FRS_TOPLEVEL)
            else:
                child = self.add_child(frs, flags)
        if not nums:
            return child
        path = [int(i) for i in nums.split(".")]
        path.reverse()
        node = child
        
        while path:
            id = path.pop()
            child = node.get(id)
            if child is None:
                child = node.add_child(id, flags)
            node = child

        return node
    
    def lookup(self, key):
        frs, nums = _frs_match(key).groups()
        node = self.get(frs)
        if not node or not nums:
            return node
        path = [int(i) for i in nums.split(".")][::-1]
        while path and node:
            id = path.pop()
            node = node.get(id)
        return node

In [393]:
KNOWN_WEBFRS_MAX = 121

class BadFRSNumber(Exception):
    pass

def load_user_test_map(ws):
    cells = ws.Cells
    cr = cells.Range
    id_start = cells.Find("ID_TEST").Offset(2, 1)
    id_end = id_start.End(xlc.xlDown)
    frs_start = cells.Find("List Web FRS").Offset(2, 1)
    frs_end = frs_start.Offset(id_end.Row - id_start.Row + 1, 1)
    test_data = cr(id_start, frs_end).Value
    test_map = []
    fixed_frs = []
    
    # since we iterate over test data here, take the opportunity to
    # convert "\n" -> "\r\n".
    for row in test_data:
        id_test, frs = row[0], row[-1]
        iid_test = int(id_test)
        if id_test == iid_test:
            id_test = iid_test
        frs = frs or ""
        frs = [f.strip() for f in frs.splitlines()]
        fixed_frs.append(("\r\n".join(frs),))
        for f in filter(None, frs):
            test_map.append((f, id_test))
    webfrs_r = cr(frs_start, frs_end)
    webfrs_r.Value = fixed_frs
        
    test_map.sort(key=sort_key)
    return test_map

def _build_tree(all_items):
    root = Root()
    for frs, flags in all_items:
        node = root.add(frs, flags)
    return root

def _add_tests(root, test_map):
    """
    :param test_map: list[(frs, id_test)]
    """
    for frs, id_test in test_map:
        node = root.lookup(frs)
        if not node:
            raise BadFRSNumber("%s %s" % (frs, id_test))
        node.add_test(id_test)

def make_frs_tree(test_map, all_items):
    root = _build_tree(all_items)
    _add_tests(root, test_map)
    return root

def _extract_frs_line(line):
    matches = (
        (_toplevel_match, FRS_TOPLEVEL),
        (_subitem_match, FRS_SUBITEM),
        (_canceled_match, FRS_NA),
    )
    for match, flags in matches:
        m = match(line)
        if m:
            return m.group(1), flags
    return None, 0
    
def load_frs_from_issuetracker():
    relevant = download_relevant_issues()
    all_frs = {(None, 0)}
    for v in relevant:
        lines = v.description.splitlines()
        for line in lines:
            frs = _extract_frs_line(line)
            all_frs.add(frs)
    for i in range(1, KNOWN_WEBFRS_MAX + 1):
        all_frs.add(("3.0WebFRS%03d" % i, FRS_TOPLEVEL))
    all_frs.remove((None, 0))
    return all_frs

def download_relevant_issues():
    api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
    issues = api.download_issues("pbssoftware")
    relevant = []
    for v in issues.values():
        if v.sprint_milestone == "3.0" and \
            v.tracker == "Specification" and \
            v.status not in ("Closed", "Rejected"):
                relevant.append(v)
    return relevant

def _xl_child_yes(row, col, n):
    form = '=IF(COUNTIF(%s,"Y")+COUNTIF(%s, "n/a")=(ROW(%s)-ROW(%s)+1), "Y", "")'
    first = row + 1, col
    last = row + n, col
    rstr = cell_range(first, last)
    r1 = cell_str(*first)
    r2 = cell_str(*last)
    return form % (rstr, rstr, r2, r1)

def make_paste_data(root):
    data = [("FRS Number", "Level", "id_test", "Tested", "Is Leaf?", "Is Holdup?")]
    di = data[0].index("Tested") + 1
    for i, node in enumerate(root.iter(), 2):
        f = node.id
        tests = "\n".join(str(i) for i in node.get_tests())
        #tested = "Y" if node.is_tested() else ""
        if node.is_leaf():
            if node.is_tested():
                tested = "Y"
            elif node.is_na():
                tested = "n/a"
            else:
                tested = ""
        else:
            if node.is_na():
                tested = "n/a"
            else:
                tested = _xl_child_yes(i, di, node.total_len())
            
        if not tests and node.has_children() and node.is_tested():
            tests = "'--"
        leaf = "Y" if node.is_leaf() else ""
        holdup = "Y" if (leaf and not tested) else ""
        data.append((f, f.count("."), tests, tested, leaf, holdup))
    return data
        
def main(user_test_matrix, unit_test_matrix=None):
    
    all_frs_items = load_frs_from_issuetracker()
    
    xl = Excel()
    wb = xl.Workbooks.Open(user_test_matrix)
    ws = wb.Worksheets("Sheet1")
    user_test_map = load_user_test_map(ws)
    
    root_reqs = make_frs_tree(user_test_map, all_frs_items)
    data = make_paste_data(root_reqs)
    
    ws2 = get_matrix_sheet(wb)
    paste_data(ws2, data)
    print("Done")
    

In [394]:
try:
    _cache
except NameError:
    _cache = None
    

def get_root_reqs(user_test_matrix):
    global _cache
    if _cache is None:
        all_frs_items = load_frs_from_issuetracker()
        _cache = all_frs_items
    else:
        all_frs_items = _cache
    
    xl = Excel()
    wb = xl.Workbooks.Open(user_test_matrix)
    ws = wb.Worksheets("Sheet1")
    user_test_map = load_user_test_map(ws)
    
    root_reqs = make_frs_tree(user_test_map, all_frs_items)
    
    
def main(user_test_matrix, unit_test_matrix=None):
    global _cache
    if _cache is None:
        all_frs_items = load_frs_from_issuetracker()
        _cache = all_frs_items
    else:
        all_frs_items = _cache
    xl = Excel()
    try:
        xl.DisplayAlerts=False
        wb = xl.Workbooks.Open(user_test_matrix)
        ws = wb.Worksheets("Sheet1")
        user_test_map = load_user_test_map(ws)

        root_reqs = make_frs_tree(user_test_map, all_frs_items)
        data = make_paste_data(root_reqs)

        ws2 = get_matrix_sheet(wb)
        paste_data(ws2, data)
        print("Done")
        return root_reqs, data
    finally:
        xl.DisplayAlerts=True

In [395]:
trace_path = "C:\\Users\\Nathan\\Documents\\PBS\\SW test\\3.0 User Test\\Traceability"
matrix_file = "Tests 161111 2.xlsx"
old_matrix = "Traceability Matrix (Reqs and VI) with full FRS check 161109.xlsx"
p1 = os.path.join(trace_path, matrix_file)
p2 = os.path.join(trace_path, old_matrix)
root, data = main(p1, p2)

Pasting test data
Applying alignment formatting
Marking untested cells
Applying column autofit
Done


In [8]:
def fix_listwebfrs():
    xl = Excel()
    for c in xl.Selection:
        v = c.Value
        if not v: continue
        c.Value = v.replace("\n", "\r\n")
#fix_listwebfrs()

In [10]:
def fix_indents():
    for c in xl.Selection:
        v = c.Value
        if v.startswith("3.0"): continue
        c.IndentLevel = v.count(".")
#fix_indents()

In [327]:

import pickle, os, datetime, difflib
from scripts.tools import issuetracker
import inspect


def clear_cache():
    global _tcache
    try:
        os.remove(_cache_file)
    except FileNotFoundError:
        pass
    _tcache = None
    
def setup():
    global IssuetrackerAPI, tests, passed, errors, _cache_file, _tcache
    tests = 0
    passed = 0
    errors = []
    _cache_file = "issues_cache.pkl"
    _tcache = None
    class MockAPI(issuetracker.IssuetrackerAPI):
        
        def __init__(self, *args, **kw):
            super().__init__(*args, login=False, **kw)

        def download_issues(self, *args, _Force=False, **kw):
   
            date, issues, reason = self._load_cache(_cache_file)
            
            if not reason:
                if _Force:
                    reason = "Forced re-cache"
                elif issues is None:
                    reason = "No existing cache found."
                elif date < datetime.date.today():
                    reason = "Old cache: %s < %s" % (date, datetime.date.today())
            
            if reason:
                print("Redownloading issues:", reason)
                issues = super().download_issues(*args, **kw)
                self._cache_issues(_cache_file, issues)
                _tcache = datetime.date.today(), issues
            return issues
        
        def _load_cache(self, file):
                global _tcache
                if _tcache is None:
                    if os.path.exists(file):
                        with open(file, 'rb') as f:
                            try:
                                date, issues = pickle.load(f)
                            except Exception as e:
                                reason = "Error: %s" % str(e)
                            else:
                                _tcache = date, issues
                                reason = ""
                    else:
                        date, issues, reason = None, None, "No Cache Found"
                else:
                    date, issues = _tcache
                    reason = ""
                return date, issues, reason   

        def _cache_issues(self, file, issues):
            date = datetime.date.today()
            with open(file, 'wb') as f:
                ob = (date, issues)
                pickle.dump(ob, f)
    IssuetrackerAPI = MockAPI
setup()
            
def fail(msg):
    global tests, errors
    tests += 1
    errors.append(msg)

def success():
    global tests, passed
    tests += 1
    passed += 1
    
def assert_equal(a, b, func=None):
    if a != b:
        if func:
            msg = func(a, b)
        else:
            msg = "%s(): %r != %r" % (inspect.stack()[1].function, a, b)
        fail(msg)
    else:
        success()
        
def get_root():
    all_frs = load_frs_from_issuetracker()
    root = _build_tree(all_frs)
    return root

def str_diff(a, b):
    return "\n".join(difflib.Differ().compare([a],[b]))
        
###########
# Tests 
###########
def test_root_all_frs():
    all_frs = load_frs_from_issuetracker()
    root = _build_tree(all_frs)
    seen = {(v.id, v.flags) for v in root.iter()}
    assert_equal(seen, all_frs)
test_root_all_frs()

def test_root_lookup():
    all_frs = load_frs_from_issuetracker()
    root = _build_tree(all_frs)
    ids = {(v.id, v.flags) for v in root.iter()}
    assert_equal(ids, all_frs)
    s1 = []
    s2 = []
    for id, flags in ids:
        s1.append((root.lookup(id).id, flags))
        s2.append((id, flags))
    assert_equal(s1, s2)
test_root_lookup()

def test_frs_strings():
    xl = Excel()
    wb = xl.Workbooks.open(test_file_path)
    ws = wb.Worksheets("Sheet1")
    test_map = load_test_map(ws)
    all_frs = load_frs_from_issuetracker()
    frs_strings = {f for f, _ in test_map}
    diff = frs_strings - all_frs
    bad_tests = {}
    for d in diff:
        tests = test_map2.get(d)
        for t in tests:
            bad = bad_tests.get(t, None)
            if bad is None:
                bad_tests[t] = bad = []
            bad.append(d)
    #     print(test_map2.get(d), d)
    for t, frs in sorted(bad_tests.items()):
        print(t, frs)
    if not bad_tests:
        print("No Bad Tests Found")
    else:
        assert False, "Bad Tests found"
        
def test_cleanup():
    canceled_match = re.compile(r"^[ \*\+]*\-.*\-").match
    relevant = download_relevant_issues()
    for v in relevant:
        lines = v.description.splitlines()
        for line in lines:
            if canceled_match(line):
                for match in (_toplevel_match, _subitem_match):
                    m = match(line)
                    if m:
                        print(line, m.groups())
                        break
                else:
                    print(line)
#test_cleanup()

def test_regex_match(s, exp, match, should_match, msg):
    m = match(s)
    if should_match:
        if not m:
            fail(msg)
            return
        assert_equal(m.group(1), exp)
    else:
        assert_equal(m, None)

def test_canceled(s, exp, match=True):
    test_regex_match(s, exp, _canceled_match, match, 
                     "%r did not match canceled regex" % s)


exp = "FRS123.4"
test_canceled("* -*FRS123.4*: bob-", exp)
test_canceled("* -*FRS123.4*-: bob", exp)
test_canceled("* -*FRS123.4*:- bob", exp)
#test_canceled("* -*FRS123.4-*: bob")
test_canceled("* *FRS123.4*: bob", exp, False)

def test_child_yes(r, c, n, exp):
    
    def on_err(a,b):
        err = "test_child_yes"
        err += "(%s, %s, %s):\n"%(r,c,n) + str_diff(a,b)
        return err
    res = _xl_child_yes(r,c,n)
    assert_equal(exp.lower(), res.lower(), on_err)
    
test_child_yes(758, 4, 4, 
               '=if(countif(D759:D762, "Y")=(ROW(D762)-ROW(D759)+1), "Y", "")')


def test_extract_frs_line(line, exp, exp_flag):
    
    def on_err(a,b):
        return "extract_frs_line(%r):\n%s" % (line, str_diff(a,b))
    
    frs = _extract_frs_line(line)
    assert_equal(frs, (exp, exp_flag), on_err)
    
test_extract_frs_line("*+FRS1234+*", "FRS1234", FRS_TOPLEVEL)
test_extract_frs_line("* *FRS1234.1:*", "FRS1234.1", FRS_SUBITEM)
test_extract_frs_line("* -*FRS123.4*: bob-", "FRS123.4", FRS_NA)
test_canceled("* -*FRS123.4*-: bob", "FRS123.4", FRS_NA)
test_canceled("* -*FRS123.4*:- bob", "FRS123.4", FRS_NA)
    
###########
# Cleanup 
###########
    
def teardown():
    global IssuetrackerAPI
    IssuetrackerAPI = issuetracker.IssuetrackerAPI
    
def finish():
    teardown()
    print("%d / %d tests passed" % (passed, tests))
    if errors:
        print("Errors found")
        for e in errors:
            print(e)
    else:
        print("Success")
finish()

13 / 13 tests passed
Success


In [281]:
def test_root_all_frs():
    all_frs = load_frs_from_issuetracker()
    root = _build_tree(all_frs)
    seen = {(v.id, v.flags) for v in root.iter()}
    print(seen-all_frs)
    print(all_frs-seen)
    
setup()
test_root_all_frs()
finish()

set()
set()
0 / 0 tests passed
Success


'_run_module_as_main'

In [2]:
import clipboard
d = clipboard.paste()
d2=[]
for line in d.splitlines():
    d2.append("** "+line.split(" ", 1)[1])
op = "\n".join(d2)
clipboard.copy(op)
print(op)

** Desktop
** Alarms Editor
** Logger Editor
** Settings Editor
** Recipe Editor
** Advanced Calibration
** DB Management
** Account Management
** Hello Interface
** Hello Access
** Remote Access
** Door
** Light
** Batch
** Recipe Start
** Recipe End
** Pumps
** Acknowledge Alarms
** Sensors
** Hello Settings
** Common
** Reports
** Controls
** System Management


In [233]:
import clipboard
t = clipboard.paste()
clipboard.copy(t.replace("shall", "will"))